In [ ]:
!pip install requests
!pip install PyYAML

In [34]:
import requests
import json
import yaml  
import logging
from pathlib import Path
from datetime import datetime

In [17]:
logging.basicConfig(
    level=logging.INFO, 
    format='%(asctime)s - %(levelname)s - %(message)s', 
    filename='Raw.log', 
    encoding='utf-8', 
    filemode='a'
)

In [18]:
logging.info("Script iniciado. Tentando carregar as configurações.")

In [19]:
try:
    with open(r"C:\Users\Carlos Vinicius\Documents\Impacta\Aulas\Python Programming for Data Engineers v2\YAML\config.yaml","r") as file:
        config=yaml.safe_load(file)
        api_key = config["api_key"]
        logging.info("Arquivo de configuração 'config.yaml' carregado com sucesso.")
except FileNotFoundError:
    logging.critical("Arquivo 'config.yaml' não foi encontrado. O programa será encerrado.")
    exit()
except KeyError:
    logging.critical("A chave 'api_key' não foi encontrada no 'config.yaml'. O programa será encerrado.")
    exit()

In [35]:
base_currency = "USD"
url = f"https://v6.exchangerate-api.com/v6/{api_key}/latest/{base_currency}"
logging.debug(f"URL da API construída: {url}")
output_dir2 = r"C:\Users\Carlos Vinicius\Documents\Impacta\Aulas\Python Programming for Data Engineers v2\Raw"
output_dir = Path(output_dir2)

In [38]:
def resultado_api(moeda, data):
    try:
        logging.info(f"Fazendo requisição para a API para a moeda base: {base_currency}")
        response = requests.get(url)
        response.raise_for_status()

        data = response.json()
        logging.info("Resposta da API recebida e decodificada com sucesso.")

        if data.get("result") == "success":
        
            try:
                # Cria um nome de arquivo com a data e hora atual para ser único
                timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
            
                filename = f"api_response_{timestamp}.json"
                full_path = output_dir / filename
            
                logging.info(f"Salvando resposta bruta da API no arquivo: {filename}")
            
                with open(full_path, 'w', encoding='utf-8') as json_file:
                    # json.dump() escreve o objeto python (data) diretamente no arquivo (json_file)
                    # indent=4 formata o arquivo para ser legível por humanos
                    # ensure_ascii=False garante que caracteres como '€' sejam salvos corretamente
                    json.dump(data, json_file, indent=4, ensure_ascii=False)
            
                logging.info(f"Dados salvos com sucesso em '{filename}'.")

            except Exception as e:
                logging.error(f"Ocorreu um erro ao tentar salvar o arquivo JSON: {e}")
            # --- FIM DA NOVA SEÇÃO ---

            # O processamento dos dados continua normalmente
            conversion_rates = data["conversion_rates"]
            logging.info("Taxas de conversão extraídas com sucesso.")

            real_brasileiro = conversion_rates["BRL"]
            euro = conversion_rates["EUR"]

            logging.info(f"Cotação encontrada: 1 {base_currency} = {real_brasileiro} BRL")
            logging.info(f"Cotação encontrada: 1 {base_currency} = {euro} EUR")

        else:
            error_type = data.get("error-type", "desconhecido")
            logging.error(f"A API retornou um erro do tipo: {error_type}")

    except requests.exceptions.RequestException:
        logging.exception("Falha na comunicação com a API. Verifique a conexão ou a URL.")
    except KeyError as e:
        logging.warning(f"A moeda {e} não foi encontrada nas taxas de conversão retornadas.")

logging.info("Script finalizado.")